In [171]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
import scipy.stats as stats
import itertools
from sklearn import metrics
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

## Spotify Calls

In [246]:
import requests
import json
import config

# retrieve access tokens

client_id = config.client_id
client_secret = config.client_secret

grant_type = 'client_credentials'

body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'

response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
response.text

'{"access_token":"BQDfybC7OtbmiUmT13i8RvfEgBBLS38iNadW2Wyb7ReDBsmHVl9wQ5q6TkYDGwKCBFU_81GFUts5VPuhCno","token_type":"Bearer","expires_in":3600,"scope":""}'

In [247]:
access_token = 'BQDfybC7OtbmiUmT13i8RvfEgBBLS38iNadW2Wyb7ReDBsmHVl9wQ5q6TkYDGwKCBFU_81GFUts5VPuhCno'
headers = {'Authorization': 'Bearer '+ access_token}

## Genres Available in Spotify's API

Before choosing what genre's will be classifiers, I make a call to check which genre's are available.

In [41]:
# retreiving spotify's list of genres
avail_genres = requests.get('https://api.spotify.com/v1/recommendations/available-genre-seeds', headers=headers)
genres = avail_genres.json()
genres

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


## Genre Playlist IDs

After choosing 15 genres, use the playlist ID to retrieve the songs available in the the genre.

In [21]:
# make a list of genre playlist IDS and names. Then convert into a dictionary

list_of_genre_ids = ['7Mr3wEdKgaiAP4Cm2a6vda', 
                     '4MRGjKqlWuJZJ8XHOGcqkR', 
                     '37i9dQZF1DX4dyzvuaRJ0n',
                     '1o1HVRGIxwCcdSnNnZ69IC',
                     '37i9dQZF1DWWEJlAGA9gs0',
                     '0TCtFMz5lY6jTfusk66ZFj',
                     '3kTtdRE1CtRyRKdicfOGAR',
                     '30BUPgw52SWNm2ZWZZc86A',
                     '6Ph1K0QWCcEwYRr0VhVt6C',
                     '5khoF3ksobwfVwOazDqpqI',
                     '6mRRGF4klfgUzbD2ZKOCq0',
                     '37i9dQZF1DWZd79rJ6a7lp',
                     '37i9dQZF1DX0xLQsW8b5Zx',
                     '2SwjQPegrTTYaOsWQrwhMe'
                    ]

list_of_genre_playlists = ['hip hop playlist',
                           'post rock playlist',
                           'electronic playlist',
                           'detroit_techo playlist',
                           'classical playlist',
                           'disco playlist',
                           'electro indie pop playlist',
                           'industrial pop playlist',
                           'french playlist',
                           'spanish playlist',
                           'ska playlist',
                           'sleep playlist',
                           'rockabilly playlist',
                           '50s Hits playlist'
                           ]

genre_dict = {list_of_genre_playlists[i]: list_of_genre_ids[i] for i in range(len(list_of_genre_playlists))}
genre_dict

{'hip hop playlist': '7Mr3wEdKgaiAP4Cm2a6vda',
 'post rock playlist': '4MRGjKqlWuJZJ8XHOGcqkR',
 'electronic playlist': '37i9dQZF1DX4dyzvuaRJ0n',
 'detroit_techo playlist': '1o1HVRGIxwCcdSnNnZ69IC',
 'classical playlist': '37i9dQZF1DWWEJlAGA9gs0',
 'disco playlist': '0TCtFMz5lY6jTfusk66ZFj',
 'electro indie pop playlist': '3kTtdRE1CtRyRKdicfOGAR',
 'industrial pop playlist': '30BUPgw52SWNm2ZWZZc86A',
 'french playlist': '6Ph1K0QWCcEwYRr0VhVt6C',
 'spanish playlist': '5khoF3ksobwfVwOazDqpqI',
 'ska playlist': '6mRRGF4klfgUzbD2ZKOCq0',
 'sleep playlist': '37i9dQZF1DWZd79rJ6a7lp',
 'rockabilly playlist': '37i9dQZF1DX0xLQsW8b5Zx',
 '50s Hits playlist': '2SwjQPegrTTYaOsWQrwhMe'}

In [32]:
# call playlist ID by calling the genre name of the playlist 
genre_dict.get('hip hop playlist')

'7Mr3wEdKgaiAP4Cm2a6vda'

## Call Functions 1.1

In [12]:
# call spotify recieve playlist of songs as dictionary
def call_spotify(genre_ID):
    i = genre_ID
    base = 'https://api.spotify.com/v1/playlists/'
    end = '/tracks'
    r = requests.get(base + i + end, headers=headers)
    songs = r.json()
    return songs

In [31]:
# example of calling spotify to retrieve all songs in a genre playlist
call_spotify(genre_dict.get('hip hop playlist'))

{'href': 'https://api.spotify.com/v1/playlists/7Mr3wEdKgaiAP4Cm2a6vda/tracks?offset=0&limit=100',
 'items': [{'added_at': '2020-01-20T12:10:39Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/redmusiccompany'},
    'href': 'https://api.spotify.com/v1/users/redmusiccompany',
    'id': 'redmusiccompany',
    'type': 'user',
    'uri': 'spotify:user:redmusiccompany'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0vRvGUQVUjytro0xpb26bs'},
       'href': 'https://api.spotify.com/v1/artists/0vRvGUQVUjytro0xpb26bs',
       'id': '0vRvGUQVUjytro0xpb26bs',
       'name': 'Arizona Zervas',
       'type': 'artist',
       'uri': 'spotify:artist:0vRvGUQVUjytro0xpb26bs'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      

## Call Functions 1.2

In [33]:
# fetch id function
def fetch_ids(songs):
    ids = []
    for song in songs['items']:
        ids.append(song['track']['id'])
    return ids

# fetch audio feature links function
def audio_features_(data):
    base =  'https://api.spotify.com/v1/audio-features/'
    list_of_ids = []
    for i in data:
        list_of_ids.append(base + str(i))
    return list_of_ids

# retrieve dictionary of track feature data
def fetch_features(links):
    feature_data = []
    for i in links:
        x = requests.get(i, headers=headers)
        j = x.content
        feature_data.append(json.loads(j))
    return feature_data

In [34]:
# combing the three functions into one

# all functions together to pull song list in and return dataframe
def song_return_feature_df(songs):
    ids = fetch_ids(songs)
    links = audio_features_(ids)
    feature_list = fetch_features(links)
    df = pd.DataFrame(feature_list)
    return df

## Call Functions 1.3

In [35]:
# call spotify playlist, return features as dataframe
def call_spotify_return_feat_df(genre_id):
    songs = call_spotify(genre_id)
    df = song_return_feature_df(songs)
    return df

In [248]:
# example of full function. Calling for song features in hip hop playlist
hip_hop_df = call_spotify_return_feat_df(genre_dict.get('hip hop playlist'))
hip_hop_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.621,0.601,6,-5.616,0,0.1480,0.05220,0.000000,0.460,0.457,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5
1,0.794,0.320,1,-12.920,0,0.1730,0.85300,0.134000,0.112,0.241,174.088,audio_features,3ClBKQkKoaUQ6UOhe2xlJK,spotify:track:3ClBKQkKoaUQ6UOhe2xlJK,https://api.spotify.com/v1/tracks/3ClBKQkKoaUQ...,https://api.spotify.com/v1/audio-analysis/3ClB...,342040,4
2,0.823,0.556,5,-5.883,0,0.1180,0.34000,0.001210,0.084,0.879,142.000,audio_features,3X6KN4WFgPYBETFMMX675P,spotify:track:3X6KN4WFgPYBETFMMX675P,https://api.spotify.com/v1/tracks/3X6KN4WFgPYB...,https://api.spotify.com/v1/audio-analysis/3X6K...,189867,4
3,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.113,0.639,136.041,audio_features,6u7jPi22kF8CTQ3rb9DHE7,spotify:track:6u7jPi22kF8CTQ3rb9DHE7,https://api.spotify.com/v1/tracks/6u7jPi22kF8C...,https://api.spotify.com/v1/audio-analysis/6u7j...,157067,4
4,0.843,0.391,2,-7.899,1,0.0845,0.18100,0.000000,0.137,0.496,129.972,audio_features,51Fjme0JiitpyXKuyQiCDo,spotify:track:51Fjme0JiitpyXKuyQiCDo,https://api.spotify.com/v1/tracks/51Fjme0Jiitp...,https://api.spotify.com/v1/audio-analysis/51Fj...,160627,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.632,0.732,6,-3.498,1,0.0920,0.10500,0.000012,0.111,0.317,159.000,audio_features,2J0NXdHr6MYvKDSxB7k3V2,spotify:track:2J0NXdHr6MYvKDSxB7k3V2,https://api.spotify.com/v1/tracks/2J0NXdHr6MYv...,https://api.spotify.com/v1/audio-analysis/2J0N...,221173,4
96,0.744,0.638,5,-4.860,0,0.0490,0.00829,0.000000,0.182,0.480,98.019,audio_features,37AGgp9VP8TVJBfEat74Bj,spotify:track:37AGgp9VP8TVJBfEat74Bj,https://api.spotify.com/v1/tracks/37AGgp9VP8TV...,https://api.spotify.com/v1/audio-analysis/37AG...,156719,4
97,0.858,0.722,10,-3.789,0,0.0766,0.02400,0.000000,0.340,0.209,143.983,audio_features,2ODfbYuZP5i2UrVXgYxPyu,spotify:track:2ODfbYuZP5i2UrVXgYxPyu,https://api.spotify.com/v1/tracks/2ODfbYuZP5i2...,https://api.spotify.com/v1/audio-analysis/2ODf...,214120,4
98,0.527,0.630,8,-7.670,1,0.0975,0.47800,0.000000,0.103,0.426,169.986,audio_features,14xJYn35NZzLEZ2D1AfGRG,spotify:track:14xJYn35NZzLEZ2D1AfGRG,https://api.spotify.com/v1/tracks/14xJYn35NZzL...,https://api.spotify.com/v1/audio-analysis/14xJ...,114360,4


In [253]:
print('Track 4 Key is', hip_hop_df['key'][4])
print('\n')
print('Track 4 Mode is', hip_hop_df['mode'][4])

Track 4 Key is 2


Track 4 Mode is 1


## Call Functions 1.4

In [42]:
# retrieving track titles from playlist
def fetch_titles(songs):
    titles = []
    for song in songs['items']:
        titles.append(song['track']['name'])
    return titles

In [49]:
# calls playlist, returns titles of songs in playlist as dataframe
def fetch_track_names(genre_id):
    songs = call_spotify(genre_id)
    titles = fetch_titles(songs)
    df = pd.DataFrame(titles)
    df = df.rename(columns={0: 'Song Titles'})
    return df

In [50]:
# example of retrieving all track names
fetch_track_names(genre_dict.get('hip hop playlist'))

,Song Titles
0,ROXANNE
1,Good News
2,NIGHTTIME (Interlude)
3,Old Town Road (feat. Billy Ray Cyrus) - Remix
4,Lalala
...,...
95,A Thousand Bad Times
96,Candy (feat. Trippie Redd)
97,By Mistake (Remix) (feat. Juicy J & Project Pat)
98,Flowers


## Call Functions 1.5

In [59]:
# retrieving track tempos 
def get_tempo(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_tempo = r_j['track']['tempo']
    return track_tempo

In [77]:
# retrieving all track tempos
def get_all_tempos():
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    tempos = []
    for i in tqdm(ids):
        tempos.append(get_tempo(i))
    return tempos

In [60]:
# retrieving track tempo confidence
def get_tempo_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    tempo_confidence = r_j['track']['tempo_confidence']
    return tempo_confidence

In [58]:
# retrieving track key
def get_key(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_key = r_j['track']['key']
    return track_key

In [76]:
# retrieving all track keys from playlist
def get_all_keys(genre_id):
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    keys = []
    for i in tqdm(ids):
        keys.append(get_key(i))
    return keys
get_all_keys(genre_dict.get('hip hop playlist'))

100%|██████████| 100/100 [01:02<00:00,  1.59it/s]


[6,
 1,
 5,
 6,
 2,
 5,
 0,
 9,
 10,
 6,
 5,
 11,
 6,
 11,
 0,
 2,
 10,
 10,
 0,
 1,
 8,
 0,
 0,
 9,
 5,
 7,
 7,
 4,
 2,
 6,
 0,
 10,
 10,
 9,
 0,
 9,
 3,
 9,
 7,
 0,
 10,
 6,
 7,
 1,
 2,
 1,
 1,
 9,
 6,
 11,
 4,
 0,
 5,
 5,
 1,
 1,
 2,
 7,
 0,
 1,
 8,
 5,
 8,
 1,
 0,
 0,
 9,
 7,
 8,
 5,
 4,
 8,
 6,
 10,
 9,
 7,
 5,
 2,
 5,
 11,
 9,
 1,
 9,
 7,
 2,
 7,
 0,
 9,
 2,
 7,
 2,
 9,
 7,
 11,
 8,
 6,
 5,
 10,
 8,
 4]

In [78]:
# retrieving track key confidence
def get_key_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    key_confidence = r_j['track']['key_confidence']
    return key_confidence

In [ ]:
# retrieving track mode of key (maj or min)
def get_key_mode(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_mode = r_j['track']['mode']
    return track_mode

In [79]:
# retrieving all track key modes from playlist
def get_all_key_modes(genre_id):
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    modes = []
    for i in tqdm(ids):
        modes.append(get_key_mode(i))
    return modes

In [80]:
# retrieveing track key mode confidence
def get_mode_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    mode_confidence = r_j['track']['mode_confidence']
    return mode_confidence

## Feature Engineering
In order to further describe how each genre might differ other than through the high level dimensions offered by Spotify's API, below are engineered features to help better describe the form of a track within a genre. The three elements of form explored are: 
    1. Harmonic Progression
    2. Modal Progression
    3. Tempo Progression
    
 These features are engineered using the audio analysis end points. Diving into the segment anaylsis and stringing each segment together to recognize the element's progression through out the track. 

In [110]:
# get audio analysis links
def audio_analysis_links(song_ids):
    base =  'https://api.spotify.com/v1/audio-analysis/'
    list_of_audio_analysis_links = []
    for i in tqdm(song_ids):
        list_of_audio_analysis_links.append(base + str(i))
    return list_of_audio_analysis_links

In [88]:
# example of retriving all audio anaylsis links for each song in playlist
# audio_analysis_links(fetch_ids(call_spotify(genre_dict.get('hip hop playlist'))))

['https://api.spotify.com/v1/audio-analysis/696DnlkuDOXcMAnKlTgXXK',
 'https://api.spotify.com/v1/audio-analysis/3ClBKQkKoaUQ6UOhe2xlJK',
 'https://api.spotify.com/v1/audio-analysis/3X6KN4WFgPYBETFMMX675P',
 'https://api.spotify.com/v1/audio-analysis/6u7jPi22kF8CTQ3rb9DHE7',
 'https://api.spotify.com/v1/audio-analysis/51Fjme0JiitpyXKuyQiCDo',
 'https://api.spotify.com/v1/audio-analysis/6fTt0CH2t0mdeB2N9XFG5r',
 'https://api.spotify.com/v1/audio-analysis/4VginDwYTP2eaHJzO0QMjG',
 'https://api.spotify.com/v1/audio-analysis/1iSqfoUFnQwV0QW1EfUit8',
 'https://api.spotify.com/v1/audio-analysis/56sxR3i2zr7AtFbgGu12ZD',
 'https://api.spotify.com/v1/audio-analysis/3VyjsVV24RmBIbWJAeUJNu',
 'https://api.spotify.com/v1/audio-analysis/5E1jEFXElio2eOxjJ3WLnk',
 'https://api.spotify.com/v1/audio-analysis/6MWtB6iiXyIwun0YzU6DFP',
 'https://api.spotify.com/v1/audio-analysis/7aiClxsDWFRQ0Kzk5KI5ku',
 'https://api.spotify.com/v1/audio-analysis/1CxYDvfq3KVvsPSZ9NCdM9',
 'https://api.spotify.com/v1/audio

In [139]:
# retrieve dictionary of track analysis data
def fetch_analysis(links):
    analysis_data = []
    count = 0
    for i in links:
        r = requests.get(i, headers=headers)
        r_j = r.content
        analysis_data.append(json.loads(r_j))
        print('--- data on link {} retrieved--'.format(count))
        count += 1
    return analysis_data


In [94]:
# fetch_analysis(audio_analysis_links(fetch_ids(call_spotify(genre_dict.get('hip hop playlist')))))

In [191]:
# keys of section (chord progression) in track
def get_harmonic_progression(song):
    section_keys = []
    for section in song['sections']:
        k = section['key']
        if k == -1:
            section_keys.append('Key not defined')
        if k == 0:
            section_keys.append('C')
        if k == 1:
            section_keys.append('C#/Db')
        if k == 2:
            section_keys.append('D')
        if k == 3:
            section_keys.append('D#/Eb')
        if k == 4: 
            section_keys.append('E')
        if k == 5: 
            section_keys.append('F')
        if k == 6:
            section_keys.append('F#/Gb')
        if k == 7:
            section_keys.append('G')
        if k == 8:
            section_keys.append('G#/Ab')
        if k == 9:
            section_keys.append('A')
        if k == 10:
            section_keys.append('A#/Bb')
        if k == 11: 
            section_keys.append('B')
    return section_keys

In [192]:
# mode of key in section (tonal progression)    
def get_harmonic_mode_progression(song):   
    section_key_modes = []
    for section in song['sections']:
        m = section['mode']
        if m == -1:
            section_key_modes.append('Mode not Defined')
        if m == 0:
            section_key_modes.append('Minor')
        if m == 1:
            section_key_modes.append('Major')
    return section_key_modes

In [193]:
# tempo changes through out sections
def get_tempo_progression(song):    
    section_tempo = []
    for section in song['sections']:
        section_tempo.append(section['tempo'])
    return section_tempo

In [194]:
# returns section info on track passed through
def get_section_info(songs):
    section_list = []
    for song in tqdm(songs):
        h_prog = get_harmonic_progression(song)
        m_prog = get_harmonic_mode_progression(song)
        t_prog = get_tempo_progression(song)
        section_prog = [h_prog, m_prog, t_prog]
        section_list.append(section_prog)
    return section_list

## Function to Output all Engineered Features

In [195]:
# calls spotify playlist, returns section info on each track in playlist
def fetch_sections_info(genre_id):
    data = call_spotify(genre_id)
    ids = fetch_ids(data)
    links = audio_analysis_links(ids)
    songs = fetch_analysis(links)
    section_info = get_section_info(songs)
    return section_info

## Example of Engineered Features

In [229]:
hip_hop_sections_df = pd.DataFrame(fetch_sections_info(genre_dict.get('hip hop playlist')))
hip_hop_sections_df = hip_hop_sections_df.rename(columns={0: 'Harmonic Progression', 
                                                          1: 'Modal Progression',
                                                          2: 'Tempo Progression'})
hip_hop_sections_df.head()





100%|██████████| 100/100 [00:00<00:00, 34903.09it/s]


--- data on link 0 retrieved--
--- data on link 1 retrieved--
--- data on link 2 retrieved--
--- data on link 3 retrieved--
--- data on link 4 retrieved--
--- data on link 5 retrieved--
--- data on link 6 retrieved--
--- data on link 7 retrieved--
--- data on link 8 retrieved--
--- data on link 9 retrieved--
--- data on link 10 retrieved--
--- data on link 11 retrieved--
--- data on link 12 retrieved--
--- data on link 13 retrieved--
--- data on link 14 retrieved--
--- data on link 15 retrieved--
--- data on link 16 retrieved--
--- data on link 17 retrieved--
--- data on link 18 retrieved--
--- data on link 19 retrieved--
--- data on link 20 retrieved--
--- data on link 21 retrieved--
--- data on link 22 retrieved--
--- data on link 23 retrieved--
--- data on link 24 retrieved--
--- data on link 25 retrieved--
--- data on link 26 retrieved--
--- data on link 27 retrieved--
--- data on link 28 retrieved--
--- data on link 29 retrieved--
--- data on link 30 retrieved--
--- data on link 3





100%|██████████| 100/100 [00:00<00:00, 36089.35it/s]

--- data on link 99 retrieved--


,Harmonic Progression,Modal Progression,Tempo Progression
0,"[D, D, A, F#/Gb, F#/Gb, A, C#/Db, F#/Gb]","[Major, Major, Major, Minor, Minor, Major, Min...","[87.699, 87.539, 92.996, 116.705, 116.841, 116..."
1,"[A, C#/Db, C#/Db, C#/Db, C#/Db, D, C#/Db, C#/D...","[Major, Minor, Minor, Minor, Minor, Major, Min...","[174.154, 174.079, 174.053, 174.844, 174.143, ..."
2,"[F, F, F, F, C#/Db, F, F, C, C#/Db]","[Minor, Minor, Minor, Minor, Major, Minor, Min...","[141.977, 141.987, 142.407, 141.945, 141.873, ..."
3,"[C#/Db, F#/Gb, C#/Db, C#/Db, F#/Gb, G#/Ab]","[Major, Major, Major, Major, Major, Minor]","[136.293, 136.264, 135.936, 135.99, 136.139, 1..."
4,"[G, D, G#/Ab, G, A, D, D, D]","[Major, Major, Major, Minor, Major, Major, Maj...","[128.952, 130.05, 129.942, 130.244, 130.035, 1..."


## Feature Engineering Function 1.2
Now that the dataframe is generated. There are a few more steps necessary before this data can be used.
    1. Clean dataframe cells so each cell has strings and not lists
    2. Transform the Harmonic Progression into Roman Numerial Analysis

In [222]:
# function to convert list of words into string
def l_to_s(_list):
    s = ''
    for i in _list:
        s = s + i + ','
    return s[:-2] 

In [223]:
# function to convert list of integers into a string
def l_to_int(_list):
    s = ''
    for i in _list:
        s = s + str(i) + ','
    return s[:-2] 

In [238]:
# cleaning
hip_hop_sections_df['Harmonic Progression'] = hip_hop_sections_df['Harmonic Progression'].apply(lambda x: list(x))
hip_hop_sections_df['Modal Progression'] = hip_hop_sections_df['Modal Progression'].apply(lambda x: l_to_s(x))
hip_hop_sections_df['Tempo Progression'] = hip_hop_sections_df['Tempo Progression'].apply(lambda x: l_to_int(x))

In [239]:
hip_hop_sections_df.head()

,Harmonic Progression,Modal Progression,Tempo Progression
0,"[D, D, A, F#/Gb, F#/Gb, A, C#/Db, F#/Gb]","Major,Major,Major,Minor,Minor,Major,Minor,Mino","87.699,87.539,92.996,116.705,116.841,116.207,1..."
1,"[A, C#/Db, C#/Db, C#/Db, C#/Db, D, C#/Db, C#/D...","Major,Minor,Minor,Minor,Minor,Major,Minor,Mino...","174.154,174.079,174.053,174.844,174.143,174.26..."
2,"[F, F, F, F, C#/Db, F, F, C, C#/Db]","Minor,Minor,Minor,Minor,Major,Minor,Minor,Mino...","141.977,141.987,142.407,141.945,141.873,141.92..."
3,"[C#/Db, F#/Gb, C#/Db, C#/Db, F#/Gb, G#/Ab]","Major,Major,Major,Major,Major,Mino","136.293,136.264,135.936,135.99,136.139,135.66"
4,"[G, D, G#/Ab, G, A, D, D, D]","Major,Major,Major,Minor,Major,Major,Major,Majo","128.952,130.05,129.942,130.244,130.035,129.87,..."


In [174]:
# read harmonic_function.py to see how Harmonic Progression is sorted
from harmonic_function import *

In [242]:
prog = get_progression_m('Major', 'G', hip_hop_sections_df['Harmonic Progression'][4])
prog

['I', 'V', 'bii', 'I', 'ii', 'V', 'V', 'V']

In [244]:
# Next I need to convert track Key in column to a pitchname
# Followed by turning track mode in column to a Major or Minor
# Modal Progression will go from 
# Major to Major
# Major to Minor Major 
# Minor to Minor
# Minor to Major to Minor

In [ ]:
# How to deal with deal with tempo progression?
# or should it be disregarded?